In [103]:
import pandas as pd
import plotly.graph_objects as go

In [104]:
database_1 = pd.read_csv('data/road_eqs_carpda_page_linear.csv')
database_2 = pd.read_csv('data/road_eqs_zev_page_linear.csv')
database_3 = pd.read_csv('data/sdg_13_10_page_linear.csv')

In [105]:
set1 = set(database_1.geo.unique())

In [106]:
set2 = set(database_2.geo.unique())

In [107]:
set3 = set(database_3.geo.unique())

In [108]:
database_3.head()

,DATAFLOW,LAST UPDATE,freq,airpol,src_crf,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:SDG_13_10(1.0),05/09/24 23:00:00,A,GHG,TOTXMEMONIA,T_HAB,AT,1990,8.9,NaN
1,ESTAT:SDG_13_10(1.0),05/09/24 23:00:00,A,GHG,TOTXMEMONIA,T_HAB,AT,1991,8.4,NaN
2,ESTAT:SDG_13_10(1.0),05/09/24 23:00:00,A,GHG,TOTXMEMONIA,T_HAB,AT,1992,8.7,NaN
3,ESTAT:SDG_13_10(1.0),05/09/24 23:00:00,A,GHG,TOTXMEMONIA,T_HAB,AT,1993,7.7,NaN
4,ESTAT:SDG_13_10(1.0),05/09/24 23:00:00,A,GHG,TOTXMEMONIA,T_HAB,AT,1994,8.7,NaN


In [109]:
database_3.src_crf.unique()

array(['TOTXMEMONIA'], dtype=object)

In [97]:
countries = set1.intersection(set2, set3)
countries

{'AT',
 'BE',
 'BG',
 'CY',
 'CZ',
 'DE',
 'DK',
 'EE',
 'EL',
 'ES',
 'EU27_2020',
 'FI',
 'FR',
 'HR',
 'HU',
 'IE',
 'IT',
 'LT',
 'LU',
 'LV',
 'MT',
 'NL',
 'PL',
 'PT',
 'RO',
 'SE',
 'SI',
 'SK'}

In [79]:
joined_df_1 = pd.merge(database_1, database_2, how='right', on=['geo', 'TIME_PERIOD'])

In [80]:
joined_df = pd.merge(joined_df_1, database_3, how='left', on=['geo', 'TIME_PERIOD'])

In [82]:
joined_df = joined_df.rename(columns={'OBS_VALUE_y':'amount_zev', 'OBS_VALUE' : 'emission_per_capita', 'OBS_VALUE_x' : 'amount_passenger_cars'})

In [110]:
joined_df
joined_df['share_zev'] = joined_df['amount_zev'] / joined_df['amount_passenger_cars']

In [84]:
groups = joined_df.groupby('geo')

# Deelvraag 1

In [85]:
for name, group in groups:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=group['TIME_PERIOD'], y=group['amount_zev'],
                             mode='lines+markers',
                             name='Amount ZEV',
                             yaxis='y1'))
    fig.add_trace(go.Scatter(x=group['TIME_PERIOD'], y=group['amount_passenger_cars'],
                             mode='lines+markers',
                             name='Amount Passenger Cars',
                             yaxis='y2'))

    fig.update_layout(
        title=f'Vehicle Data for {name}',
        xaxis_title='Time Period',
        yaxis_title='Amount ZEV',
        yaxis=dict(
            title='Amount ZEV',
            side='left',
            showgrid=False
        ),
        yaxis2=dict(
            title='Amount Passenger Cars',
            side='right',
            overlaying='y',
            showgrid=False
        ),
        legend_title='Legend'
    )

    fig.show()

# Deelvraag 2

In [99]:
for name, group in groups:
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=group['TIME_PERIOD'], y=group['emission_per_capita'],
                             mode='lines+markers',
                             name='Emission per Capita',
                             yaxis='y1'))

    fig.add_trace(go.Scatter(x=group['TIME_PERIOD'], y=group['share_zev'],
                             mode='lines+markers',
                             name='Share ZEV',
                             yaxis='y2'))

    fig.update_layout(
        title=f'Environmental Data for {name}',
        xaxis_title='Time Period',
        yaxis=dict(
            title='Emission per Capita',
            side='left',
            showgrid=False,
            type='linear'
        ),
        yaxis2=dict(
            title='Share ZEV',
            side='right',
            overlaying='y',
            showgrid=False,
            type='linear'
        ),
        legend_title='Legend'
    )


    fig.show()

# Deelvraag 3

In [111]:
fig = go.Figure()

for name, group in groups:
    fig.add_trace(go.Scatter(
        x=group['TIME_PERIOD'],
        y=group['share_zev'],
        mode='lines+markers',
        name=name
    ))

fig.update_layout(
    title='Comparison of Share ZEV over Time by Geo Group',
    xaxis_title='Time Period',
    yaxis_title='Share ZEV',
    legend_title='Geo Group'
)

fig.show()